In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

###### spark 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

###### Загрузка датасета 

In [3]:
data = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET, hive)

In [4]:
dataset = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET, hive).cache()
dataset_positive = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_POSITIVE, hive).cache()

###### create story target items 

In [5]:
item_dict = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_ITEM_DICT, hive)

In [6]:
item_dict.filter("product <> '' and product <> 'none'").show(3)

,hitPagePath,eventCategory,eventAction,product,event_id,event_user_count,event_user_percent,prod_id,prod_user_count,prod_user_percent,item_id,item_user_count,item_user_percent
0,/shop/activated-products,[std]: shop,click,accounts_and_payments,105012110,9875,0.95079,214195630,1599,0.15396,214195630,1599,0.15396
1,/shop/activated-products,[std]: shop,click,api,105012110,9875,0.95079,214197701,14,0.00135,214197701,14,0.00135
2,/shop/all-products,[std]: shop,click,api,105028375,43083,4.14814,214197701,14,0.00135,214197701,14,0.00135


In [7]:
product_item = item_dict.filter("product <> '' and product <> 'none'").select("item_id", "product", "item_user_count").distinct()

Выделим целевые события для скоринга

In [8]:
product_list = ['leasing',
                'guarantee',
                'rabota_ru',
                'merchant-acquiring',
                'zarplata',
                'korpkarta',
                'two_gis',
                'evotor',
                'e-acquiring'
                'mybuch',
                'sberrating',
                'credit_card',
                'mybuch_online_ip',
                'lyurist',
                'kredit']

In [9]:
target_items = product_item.filter(f.col("product").isin(product_list)).toPandas()["item_id"].to_list()

In [10]:
len(target_items)

13

In [12]:
item_dict.filter(f.col("item_id").isin(target_items)) \
            .groupBy("item_id").agg(f.first("product"), f.first("item_user_count")).show(13)

,item_id,"first(product, false)","first(item_user_count, false)"
0,214250791,mybuch_online_ip,296
1,214300159,rabota_ru,2849
2,214230946,kredit,5203
3,214238693,leasing,449
4,214314599,sberrating,2606
5,214337742,two_gis,308
6,214228698,korpkarta,1802
7,214240895,lyurist,2362
8,214204148,credit_card,2303
9,214224178,evotor,823


###### Разделение пользователей

In [13]:
users = dataset_positive.select("user_id").distinct()
users.count()

969905

###### Пользователи с целевыми событиями в кликстриме

In [14]:
users_with_target = dataset_positive.filter(f.col("item_id").isin(target_items)) \
                                    .select("user_id").distinct()
users_with_target.count()

22746

###### Пользователи без целевых событий в кликстриме

In [15]:
users_without_target = users.subtract(users_with_target)
users_without_target.count()

947159

Датасет для этих пользователей => в обучающую выборку

In [16]:
dataset_train_without_target = dataset_positive.join(users_without_target, on="user_id", how="inner")
dataset_train_without_target.count()

8315433

###### Разделение пользователей с целевыми событиями

In [17]:
users_train, users_val_test = users_with_target.randomSplit([0.8, 0.2])
print("Train: {}".format(users_train.count()))
print("Val/test: {}".format(users_val_test.count()))

Train: 18101
Val/test: 4645


###### Обучающая выборка

In [18]:
dataset_train_with_target = dataset_positive.join(users_train, on="user_id", how="inner")
dataset_train_with_target.count()

396624

Объединяем пользователей с нецелевыми событиями с обучающей частью с целевыми

In [19]:
dataset_train = dataset_train_with_target.union(dataset_train_without_target)
dataset_train.count()

8712057

###### Отсечение по времени первого целевого действия

Установка времени первого целевого события

In [20]:
users_val_test_min_target_time = dataset_positive.filter(f.col("item_id").isin(target_items)) \
                                                .join(users_val_test, on="user_id", how="inner") \
                                                .groupBy("user_id").agg(f.min("timestamp").alias("min_target_timestamp"))
users_val_test_min_target_time.show(3)

,user_id,min_target_timestamp
0,140021,1622718967
1,333479,1623340026
2,397863,1623113885


###### Отсечение

Обнуляем все события, которые произошли после первого целевого (включая)

In [21]:
dataset_val_test = dataset_positive.join(users_val_test_min_target_time, on="user_id", how="inner") \
                                    .withColumn("new_rating", f.when(f.col("timestamp") >= f.col("min_target_timestamp"), 0) \
                                                               .otherwise(f.col("rating"))) \
                                    .drop("rating", "min_target_timestamp") \
                                    .withColumnRenamed("new_rating", "rating")

dataset_val_test_corr = dataset_val_test.filter("rating = 1")

Среднее количество эвентов на пользователя до и после отсечения

In [22]:
print("Avg events: {}".format(dataset_val_test.groupBy("user_id").count() \
                                              .select(f.avg("count").alias("avg")) \
                                              .toPandas().avg.values[0]))

print("Avg new events: {}".format(dataset_val_test_corr.groupBy("user_id").count() \
                                                      .select(f.avg("count").alias("avg")) \
                                                      .toPandas().avg.values[0]))

Avg events: 21.629063509149624
Avg new events: 11.565084586466165


In [23]:
print("Event count: {}".format(dataset_val_test.count()))
print("New event count: {}".format(dataset_val_test_corr.count()))

Event count: 100467
New event count: 49221


###### Добавление скорректированной выборки в датасет

In [24]:
dataset_val_test = dataset_val_test_corr.join(users_val_test, on="user_id", how="inner")

dataset_full = dataset_train.union(dataset_val_test)
dataset_full.count()

8761278

Пользователи, у которых первое событие в кликстриме целевое - не попадают в финальную выборку

In [25]:
delta_users = dataset_positive.select("user_id").distinct() \
                                .subtract(dataset_full.select("user_id").distinct())
delta_users.count()

389

Удаляем этих пользователей из списков на валидацию и тест

In [26]:
users_val_test_corr = users_val_test.subtract(delta_users)

###### Разделение на валидацию и тест

In [27]:
users_val, users_test = users_val_test_corr.randomSplit([0.5, 0.5])

###### Формирование меток для предсказания из целевых эвентов

In [28]:
target_val = dataset.filter(f.col("item_id").isin(target_items)) \
                    .join(users_val, on="user_id", how="inner")

target_test = dataset.filter(f.col("item_id").isin(target_items)) \
                     .join(users_test, on="user_id", how="inner")

###### Сохранение данных

In [29]:
drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_TRAIN_POS, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_TRAIN_POS, dataset_full, hive)

In [30]:
drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_VAL, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_VAL, target_val, hive)

In [31]:
drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_TEST, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_TEST, target_test, hive)

###### проверка сохранения таблиц 

In [32]:
train_sdf = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_TRAIN_POS, hive)
train_sdf.show()

,user_id,item_id,timestamp,rating
0,24347,50218704,1623244974,1.0
1,24347,50479022,1622688044,1.0
2,24347,50165164,1622465176,1.0
3,24347,95542157,1622515893,1.0
4,24347,81355473,1622698694,1.0
5,24347,105401066,1622859303,1.0
6,24347,84853379,1622516041,1.0
7,24347,58109115,1622697828,1.0
8,24347,105921618,1622620654,1.0
9,24347,63758127,1622688035,1.0


In [33]:
load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_VAL, hive).filter("rating = 1").count()

2439

In [34]:
load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_TEST, hive).filter("rating = 1").count()

2482